In [88]:
# Import libraries and dependencies
import os
import numpy as np
import pandas as pd
import alpaca_trade_api as tradeapi
import plotly.graph_objects as go

# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [90]:
pip install polars


   ---------------------------------------- 0.0/27.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/27.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/27.2 MB 330.3 kB/s eta 0:01:23
   ---------------------------------------- 0.0/27.2 MB 330.3 kB/s eta 0:01:23
   ---------------------------------------- 0.1/27.2 MB 469.7 kB/s eta 0:00:58
   ---------------------------------------- 0.2/27.2 MB 766.6 kB/s eta 0:00:36
   ---------------------------------------- 0.2/27.2 MB 962.7 kB/s eta 0:00:29
   ---------------------------------------- 0.2/27.2 MB 962.7 kB/s eta 0:00:29
   ---------------------------------------- 0.2/27.2 MB 962.7 kB/s eta 0:00:29
   ---------------------------------------- 0.2/27.2 MB 962.7 kB/s eta 0:00:29
    --------------------------------------- 0.4/27.2 MB 849.3 kB/s eta 0:00:32
    --------------------------------------- 0.5/27.2 MB 1.1 MB/s eta 0:00:25
   - -------------------------------------- 0.9/27.2 MB 1.6 MB/s eta 0:

In [107]:

# ### Get 4 Years Worth of Data via API Call and Read in as DataFrame 
# # Set the ticker
# tickers = ['VTI','VEA','VWO','BND','VNQ','BNDX']

# # Set timeframe to "1Day"
# timeframe = "1Day"

# # Set start and end datetimes of 4 years from Today
# start_date = pd.Timestamp("2014-05-04", tz="America/New_York").isoformat()
# end_date = pd.Timestamp("2024-04-04", tz="America/New_York").isoformat()

# all_ticker_data = pd.DataFrame()

# for ticker in tickers:
#     # Get historical data for AAPL
#     ticker_data = api.get_bars(
#         ticker,
#         timeframe,
#         start=start_date,
#         end=end_date
#     ).df

#     ticker_data = pd.DataFrame(ticker_data['close'].pct_change().cumsum())
#     ticker_data = ticker_data.rename(columns={'close':ticker})

#     # ticker_data.head()
#     # Reorganize the DataFrame
#     all_ticker_data = pd.concat([all_ticker_data, ticker_data], axis=1)

# # Display sample data
# all_ticker_data = all_ticker_data.reset_index()
# all_ticker_data.head()


import polars as pl
from datetime import datetime
import pytz  # For timezone handling if required

# Set the ticker
tickers = ['VTI', 'VEA', 'VWO', 'BND', 'VNQ', 'BNDX']

# Set timeframe to "1Day"
timeframe = "1Day"

# Set start and end datetimes of 4 years from Today
# Adjust timezone if necessary
start_date = datetime(2014, 5, 4, tzinfo=pytz.timezone("America/New_York"))
end_date = datetime(2024, 4, 4, tzinfo=pytz.timezone("America/New_York"))

all_ticker_data = pl.DataFrame()

for ticker in tickers:
    # Get historical data for the ticker
    ticker_data = api.get_bars(
        ticker,
        timeframe,
        start=start_date.isoformat(),
        end=end_date.isoformat()
    ).df  # Assuming the API returns a DataFrame; convert to Polars DataFrame

    # Reset the index to make sure datetime becomes a column
    ticker_data = ticker_data.reset_index()

    # Convert pandas DataFrame to Polars DataFrame
    ticker_data = pl.from_pandas(ticker_data)

    # Calculate cumulative sum of percentage change and keep the 'timestamp' column
    ticker_data = ticker_data.select([
        pl.col('timestamp'),
        pl.col('close').pct_change().cum_sum().alias(ticker)
    ])

    # If all_ticker_data is empty, initialize it with ticker_data
    if all_ticker_data.width == 0:
        all_ticker_data = ticker_data
    else:
        # Otherwise, join the new ticker_data to the existing DataFrame
        # and handle the suffix issue by using custom suffixes or removing unnecessary columns
        all_ticker_data = all_ticker_data.join(
            ticker_data, on='timestamp', how='outer',
            suffix="_right"  # You can specify custom suffixes if needed
        ).drop(['timestamp_right'])  # Drop the right timestamp if it's created

# Reset index if needed and show the first few rows
all_ticker_data.head()



timestamp,VTI,VEA,VWO,BND,VNQ,BNDX
"datetime[ns, UTC]",f64,f64,f64,f64,f64,f64
2016-01-04 05:00:00 UTC,null,null,null,null,null,null
2016-01-05 05:00:00 UTC,0.002239,-0.001106,0.002203,0.001177,0.019297,-0.000566
2016-01-06 05:00:00 UTC,-0.011163,-0.019373,-0.015694,0.005631,0.016557,0.00321
2016-01-07 05:00:00 UTC,-0.035772,-0.038262,-0.047983,0.005754,-0.002926,0.001705
2016-01-08 05:00:00 UTC,-0.047075,-0.050619,-0.056902,0.006986,-0.0163,0.001517


In [108]:
fig = go.Figure()

for ticker in tickers:
    fig.add_trace(go.Scatter(x=all_ticker_data['timestamp'],y=all_ticker_data[ticker], name=ticker))
fig.show()